In [40]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from splinter import Browser
import sys
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
import tweepy
from datetime import datetime
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

### Mars news

In [2]:
#NASA news site, pull latest article text and date
url = "https://mars.nasa.gov/api/v1/news_items/?order=publish_date+desc"
response = requests.get(url).json()
body = response.get('items')[0]




In [39]:
url= "https://mars.nasa.gov/api/v1/news_items/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
news = requests.get(url)
news = news.json()
most_recent = news.get('items')[0]
most_recent_b = most_recent['body']
most_recent_t =most_recent['title']

soup = BeautifulSoup(most_recent_b, 'html.parser')
ps = soup.find_all('p')
most_recent_body = ps[1].get_text()

{"title": most_recent_t, "body": most_recent_body}


{'body': 'No one will know how the rover is doing until it speaks. But the team notes there’s reason to be optimistic: They’ve performed several studies on the state of its batteries before the storm, and temperatures at its location. Because the batteries were in relatively good health before the storm, there’s not likely to be too much degradation. And because dust storms tend to warm the environment -- and the 2018 storm happened as Opportunity’s location on Mars entered summer -- the rover should have stayed warm enough to survive.',
 'title': "Six Things About Opportunity's Recovery Efforts"}

In [35]:
soup = BeautifulSoup(most_recent_b, 'html.parser')
ps = soup.find_all('p')
ps[1].get_text()

'No one will know how the rover is doing until it speaks. But the team notes there’s reason to be optimistic: They’ve performed several studies on the state of its batteries before the storm, and temperatures at its location. Because the batteries were in relatively good health before the storm, there’s not likely to be too much degradation. And because dust storms tend to warm the environment -- and the 2018 storm happened as Opportunity’s location on Mars entered summer -- the rover should have stayed warm enough to survive.'

### mars image

In [43]:
#soup to find image url
img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
image_pg = requests.get(img_url)
soup = BeautifulSoup(image_pg.text, "html.parser")
img_link = soup.find("a", {"class":"button fancybox"}).attrs.get('data-fancybox-href')
# feature_image = img_url + img_link
feature_image = f"https://www.jpl.nasa.gov{img_link}"
feature_image

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18249_ip.jpg'

### mars weather twitter

In [5]:
from config import *

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(app_key, app_secret)
api = tweepy.API(auth)

In [6]:
#pull latest 10 tweets, convert to json, read 1st tweet
mars_tweets = []

for status in tweepy.Cursor(api.user_timeline, id="@MarsWxReport").items(10):
    mars_tweets.append(status)

mars_weather = mars_tweets[0]._json


In [26]:
#pull text/status update of the latest tweet
tweet_text = mars_weather['text']

#date/time of tweet
date = mars_weather['created_at']
tweet_date = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(date,'%a %b %d %H:%M:%S +0000 %Y'))

# print(f"On {tweet_date} the weather on mars was {tweet_text}")

tweet = {"date": date, "tweet":tweet_text }
tweet

{'date': 'Sun Aug 12 08:59:51 +0000 2018',
 'tweet': 'RT @rtphokie: A view of this morning’s launch of the Parker #SolarProbe launch from about 3 miles away \nhttps://t.co/ulKl6gldeR'}

### Mars Facts

In [42]:
mars_fact_url = "https://space-facts.com/mars/"

#read html table through pandas
mars_facts = pd.read_html(mars_fact_url)
mars_facts_df=pd.DataFrame(mars_facts[0])

#rename columns
mars_facts_df.columns=["Description", "Value"]
mars_facts_df = mars_facts_df.set_index("Description").reset_index()
mars_facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
#convert table to HTML string for web
import numpy as np
mars_facts_df.to_html('mars_table.html')


### mars hemisphere images

In [11]:
#pull high res images from astrology site
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base_hems_url= "https://astrogeology.usgs.gov"
hems = requests.get(hemisphere_url)
hems_soup = BeautifulSoup(hems.text, "html.parser")

hemispheres = hems_soup.find_all("a", {"class": "product-item"})

#empty list to hold image name/url
mars_hemispheres=[]

for hemi in hemispheres:
    img_title = hemi.find("h3").get_text().replace("Enhanced", "")#remove enhanced text from image title
        #remove enhanced text from image title
        # create full image url
    img_link = base_hems_url + hemi.attrs.get('href') 
        # access the downloads page for full high res image
    high_res_soup = BeautifulSoup(requests.get(img_link).text, "html.parser")
    download = high_res_soup.find("div", {"class":"downloads"})
    img_url = download.find('a')['href']
    mars_hemispheres.append({"title": img_title, "img_url": img_url})

mars_hemispheres  
    

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere '}]